In [ ]:
from ratdata import data_manager as dm, process, ingest
import scipy.signal as signal
import pathlib

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
recording_files = dm.RecordingFile.select()

In [ ]:
recalculate = True
scale = 1
beta_low = 14
beta_high = 18
oof_fit_low = 2
oof_fit_high = 100
total_low = 10
total_high = 100
target_fs = 500
length_seconds = 5
f_b, f_a = signal.iirnotch(50, 30, target_fs)

for i, f in enumerate(recording_files):
    if recalculate or (f.slice.count() == 1 and f.slice.get().updated):
        fname = pathlib.Path(f.dirname) / f.filename
        recording = ingest.read_mce_matlab_file(str(fname))
        fs = int(1 / recording.dt)
        if f.slice.count() == 1:
            slice = f.slice.get()
            data, tt = process.trim_recording(recording.electrode_data.mean(0), fs, slice.start, slice.length)
        else:
            data = recording.electrode_data.mean(0)
        # Decimate
        data = signal.decimate(data, int(fs / target_fs))
        # Reject 50 Hz
        data = signal.filtfilt(f_b, f_a, data)
        # Select first 5 s
        data = data[:length_seconds * target_fs]
        recording_beta_power = process.power_in_frequency_band(data, beta_low, beta_high, target_fs)
        recording_total_power = process.power_in_frequency_band(data, total_low, total_high, target_fs)
        ff, pxx = signal.welch(data, target_fs, nperseg=2 * target_fs)
        m, b = process.fit_oof(ff, pxx, oof_fit_low, oof_fit_high)
        rp_id = dm.upsert_power_record(f, recording_beta_power, recording_total_power, m, b)
        print('[%d/%d] %s: %d' % (i + 1, len(recording_files), f.filename, rp_id))
    